# Part 1

1. **Load data**
    1. sampling rate 지정 (4kHz)
        
    2. librosa.load 사용
    

2. **Pre-processing**
    1. raw data cutting
        - 한 호흡 주기에 맞춰서 data 자르기


3. **Feature Extraction**
    -  Sound data 특징 추출 및 이미지화 : Mel - spectrogram
    



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns

import librosa as lb
import librosa.display as lbd
import os


In [ ]:
train=pd.read_csv('/content/drive/MyDrive/aiffel/aiffelthon/csv_data/train.csv')
val=pd.read_csv('/content/drive/MyDrive/aiffel/aiffelthon/csv_data/val.csv')
train.head() #((5173, 3), (1725, 3))

,Unnamed: 0,pid,filename,disease
0,1572,132,132_2b2_Lr_mc_LittC2SE_1.wav,COPD
1,2266,143,143_1b1_Al_sc_Meditron_17.wav,Healthy
2,561,154,154_4b4_Pr_mc_AKGC417L_4.wav,COPD
3,3165,209,209_1b1_Tc_sc_Meditron_2.wav,Healthy
4,1349,130,130_2b2_Ll_mc_AKGC417L_7.wav,COPD


In [ ]:
train.shape

(5173, 4)

In [ ]:
ytrain=train.disease
yval=val.disease
yval

0       COPD
1       COPD
2       COPD
3       COPD
4       COPD
        ... 
1720    COPD
1721    COPD
1722    COPD
1723    COPD
1724    COPD
Name: disease, Length: 1725, dtype: object

* As our class labels are of **dtype: object** we must first convert them to **encoded values** or intergers

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
ytrain=le.fit_transform(ytrain)
yval=le.transform(yval)

# Feature Extraction

> I am going to use Librosa's feature extraction methods here

> Note:- i first tried to extract features **1-by-1** and then test them, to see how they perform but most of then were giving **good accuracy** so i finally decided to use all of them



In [ ]:
def getFeatures(path):
    soundArr,sample_rate=lb.load(path)
    mfcc=lb.feature.mfcc(y=soundArr,sr=sample_rate)
    cstft=lb.feature.chroma_stft(y=soundArr,sr=sample_rate)
    mSpec=lb.feature.melspectrogram(y=soundArr,sr=sample_rate)

    return mfcc,cstft,mSpec

> In These functions i m iterating over **train and val dataset**, and using **filenames** to extract features and then converting them to **numpy arrays**


In [ ]:
root='/content/drive/MyDrive/aiffel/aiffelthon/processed_audio_files/'
mfcc_v,cstft_v,mSpec_v=[],[],[]
i=0
for idx,row in val.iterrows():
    path=root + row['filename']
    a,b,c=getFeatures(path) #wavform 하나 당 3개 피처 뽑아서 리스트에 추가
    mfcc_v.append(a)
    cstft_v.append(b)
    mSpec_v.append(c)
    


/usr/local/lib/python3.7/dist-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")


In [ ]:
mfcc_val=np.array(mfcc_v) # ndarray 배열로, 나중에 텐서로 바꾸나?
cstft_val=np.array(cstft_v)
mSpec_val=np.array(mSpec_v)

np.savez('/content/drive/MyDrive/aiffel/aiffelthon/features/features_val.npz',mfcc_val=mfcc_val, cstft_val=cstft_val, mSpec_val=mSpec_val)

The shape of features returned by the above function **(20, 259) (12, 259) (128, 259)** we need this for defining **input shape** of our model.

In [ ]:
mfcc_val.shape

(5173, 20, 259)

In [ ]:
root='/content/drive/MyDrive/aiffel/aiffelthon/processed_audio_files/'
mfcc,cstft,mSpec=[],[],[]
i=0
for idx,row in train.iterrows():
    path=root + row['filename']
    a,b,c=getFeatures(path)
    mfcc.append(a)
    cstft.append(b)
    mSpec.append(c)
    


/usr/local/lib/python3.7/dist-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")


In [ ]:
mfcc_train=np.array(mfcc)
cstft_train=np.array(cstft)
mSpec_train=np.array(mSpec)

np.savez('/content/drive/MyDrive/aiffel/aiffelthon/features/features_train.npz',mfcc_train=mfcc_train, cstft_train=cstft_train, mSpec_train=mSpec_train)

> Discarded few features beacuse they were not important to performance

In [ ]:
print(mfcc_train.shape)
print(cstft_train.shape)
print(mSpec_train.shape)

(5173, 20, 259)
(5173, 12, 259)
(5173, 128, 259)
